In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import pairwise_distances
from scipy.spatial import distance_matrix

#from sklearn import cluster
#from sklearn.preprocessing import StandardScaler
%matplotlib inline


$$
\begin{aligned}
&\begin{array}{|c|c|c|}
\hline
\hline \textbf { Constantes} & \textbf {  Funciones } & \textbf { Clases }  \\
\hline
\text{Genome ids (list)} &  &  \\
\hline
\text{Family ids (list o dict)} &  & \\
\hline
\text{NP ids}  &  &  \\
\hline
\text{Tamaños de los dataframes} & & \\
\end{array}
\end{aligned}
$$

In [17]:
df1 = pd.read_csv("/home/csar/Proyectos/Posdoc/Proyecto_pos/CsarPos/blast_EF_vs_GenomeDB/pscplos17.blast", sep = "\t", names = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'], engine = 'python')

In [18]:
DF1 = df1[df1["bitscore"]>=100]
DF1.reset_index(drop=True, inplace=True)
DF1

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,79.62,530,108,0,1,530,1,530,0.000000e+00,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,59.59,532,213,2,1,530,1,532,0.000000e+00,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,59.20,527,214,1,1,526,1,527,0.000000e+00,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,59.20,527,214,1,1,526,1,527,0.000000e+00,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,60.34,532,209,2,1,530,1,532,0.000000e+00,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...
277,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146852.774|6666666.146852|NC_002935...,61.33,362,139,1,1,362,11,371,5.000000e-165,471.0
278,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146872.438|6666666.146872|NC_017803...,60.06,363,145,0,1,363,12,374,9.000000e-165,471.0
279,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.111572.2478|6666666.111572|NC_00948...,64.64,362,126,2,1,362,302,661,1.000000e-156,460.0
280,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|367928.21.407|367928.21|NC_008618.1|Phospho...,57.30,370,151,2,1,363,11,380,6.000000e-150,433.0


In [19]:
def extract_GenomeInfo(data):
    genomas_id = []
    genes_id = []
    genomas_nombres = []
    for x in data:
        x_nombre = x.split("|")[-1]
        x_ID = x.split("|")[1]
        x_gi = x_ID.split(".")[0]+"."+ x_ID.split(".")[1]
        x_gen = x_ID.split(".")[2]
        genomas_nombres.append(x_nombre)
        genomas_id.append(x_gi)
        genes_id.append(x_gen)
    
    genome_info = {"genome_name": genomas_nombres, "id_genome": genomas_id, "id_gene": genes_id}
    genome_info = pd.DataFrame(genome_info, columns = ["genome_name", "id_genome", "id_gene"])
    return genome_info


DF1 = pd.concat([DF1,extract_GenomeInfo(DF1["subject"])], axis=1)
DF1    

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore,genome_name,id_genome,id_gene
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,79.62,530,108,0,1,530,1,530,0.000000e+00,847.0,CorynebacteriumdiphtheriaeNCTC13129s,6666666.146852,1090
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,59.59,532,213,2,1,530,1,532,0.000000e+00,620.0,AmycolatopsismediterraneiU32s,749927.13,1727
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,59.20,527,214,1,1,526,1,527,0.000000e+00,613.0,AmycolatopsisorientalisHCCB10007s,1156913.7,6305
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,59.20,527,214,1,1,526,1,527,0.000000e+00,612.0,AmycolatopsisdecaplaninaDSM44594s,6666666.104540,4265
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,60.34,532,209,2,1,530,1,532,0.000000e+00,595.0,ActinosynnemamirumDSM43827NC0130931s,446462.15,6083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146852.774|6666666.146852|NC_002935...,61.33,362,139,1,1,362,11,371,5.000000e-165,471.0,CorynebacteriumdiphtheriaeNCTC13129s,6666666.146852,774
278,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146872.438|6666666.146872|NC_017803...,60.06,363,145,0,1,363,12,374,9.000000e-165,471.0,ActinoplanesspSE50110s,6666666.146872,438
279,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.111572.2478|6666666.111572|NC_00948...,64.64,362,126,2,1,362,302,661,1.000000e-156,460.0,ClavibactermichiganensissubspmichiganensisNCPP...,6666666.111572,2478
280,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|367928.21.407|367928.21|NC_008618.1|Phospho...,57.30,370,151,2,1,363,11,380,6.000000e-150,433.0,BifidobacteriumadolescentisATCC15703s,367928.21,407


**Función para obtener las semillas y familias de las enzimas**

In [20]:
def extract_fam_seed(data):
    fam_list = []
    seed_list = []
    for x in data:
        x_string = x.split("|")
        x_fam = x_string[1]
        x_enz = x_string[2].split("_")[-1]
        #x_value = "F"+x_fam+"S"+x_enz
        fam_list.append(x_fam)
        seed_list.append(x_enz)
    enzime_info = {"fam": fam_list, "seed": seed_list}
    enzime_info = pd.DataFrame(enzime_info, columns=["fam", "seed"])
    return enzime_info
DF1 = pd.concat([DF1,extract_fam_seed(DF1["query"])], axis=1)
DF1

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore,genome_name,id_genome,id_gene,fam,seed
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,79.62,530,108,0,1,530,1,530,0.000000e+00,847.0,CorynebacteriumdiphtheriaeNCTC13129s,6666666.146852,1090,1,1
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,59.59,532,213,2,1,530,1,532,0.000000e+00,620.0,AmycolatopsismediterraneiU32s,749927.13,1727,1,1
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,59.20,527,214,1,1,526,1,527,0.000000e+00,613.0,AmycolatopsisorientalisHCCB10007s,1156913.7,6305,1,1
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,59.20,527,214,1,1,526,1,527,0.000000e+00,612.0,AmycolatopsisdecaplaninaDSM44594s,6666666.104540,4265,1,1
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,60.34,532,209,2,1,530,1,532,0.000000e+00,595.0,ActinosynnemamirumDSM43827NC0130931s,446462.15,6083,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146852.774|6666666.146852|NC_002935...,61.33,362,139,1,1,362,11,371,5.000000e-165,471.0,CorynebacteriumdiphtheriaeNCTC13129s,6666666.146852,774,2,3
278,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146872.438|6666666.146872|NC_017803...,60.06,363,145,0,1,363,12,374,9.000000e-165,471.0,ActinoplanesspSE50110s,6666666.146872,438,2,3
279,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.111572.2478|6666666.111572|NC_00948...,64.64,362,126,2,1,362,302,661,1.000000e-156,460.0,ClavibactermichiganensissubspmichiganensisNCPP...,6666666.111572,2478,2,3
280,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|367928.21.407|367928.21|NC_008618.1|Phospho...,57.30,370,151,2,1,363,11,380,6.000000e-150,433.0,BifidobacteriumadolescentisATCC15703s,367928.21,407,2,3


In [93]:
DF1[(DF1["fam"]=="1")&(DF1["genome_name"]=="CatenulisporaacidiphilaDSM44928s")]["id_gene"].drop_duplicates()

6      7934
35     6857
36     3876
42      130
47      635
53     6960
59     5905
62     2729
112    5387
Name: id_gene, dtype: object

**Función para obtener las copias dentro de cada génoma.

Argumento: -data -> DataFrame DF1 que contiene solo aquellos genomas con bitscore >=100.
           -fam_id -> número que identifica a la familia
           -GenomeNames -> Lista con los nombres de los genomas.

Salida: Diccionario con los genomas como claves, y el valor de cada clave es una lista con los id
        de las copias de las semillas de la Familia F dentro del genoma da nombre a la clave. 
        

In [117]:
GenomeNames = DF1["genome_name"].drop_duplicates()
genome_dict = {}
for name in GenomeNames:
    genome_dict[name] = []
help(genome_dict)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Built-in subclasses:
 |      StgDict
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>va

In [108]:
x ={}
for i in range(5):
    x[i] = []
print(x)
for i in range(5):
    x[i].append(i-1)
print(x)
    

{0: [], 1: [], 2: [], 3: [], 4: []}
{0: [-1], 1: [0], 2: [1], 3: [2], 4: [3]}


In [111]:

def gene_copies(data, fam_id):   #dataframe DF1 y número de identificación de la familia
    gene_dict = genome_dict
    L = len(data)
    for i in range(L):
        if (data["fam"][i]==fam_id):
            gene_dict[data["genome_name"][i]].append(data["id_gene"][i])
    return gene_dict

gene_copies(DF1, "1")
        
            

AttributeError: 'NoneType' object has no attribute 'append'

In [91]:
DF1[["fam","genome_name","id_gene"]].drop_duplicates().value_counts(["fam", "genome_name", "id_gene"]).head(20)


fam  genome_name                            id_gene
1    ActinoplanesspN902109s                 1388       1
                                            2685       1
     CatenulisporaacidiphilaDSM44928s       6857       1
                                            635        1
                                            5905       1
                                            5387       1
                                            3876       1
                                            2729       1
                                            130        1
     Bifidobacteriumanimalissubsplactiss    829        1
                                            379        1
     BifidobacteriumadolescentisATCC15703s  325        1
                                            1188       1
                                            1038       1
     ArthrobacterphenanthrenivoransSphe3s   608        1
                                            2378       1
     ArthrobacterchlorophenolicusA6s

In [84]:
DF1[["fam","genome_name","id_gene"]].drop_duplicates().value_counts(["fam", "genome_name"]).to_frame("copy_number")


copy_number
fam genome_name                                                    
1   CatenulisporaacidiphilaDSM44928s                              9
    AmycolatopsismediterraneiU32s                                 8
    ArthrobacterchlorophenolicusA6s                               8
    AmycolatopsisorientalisHCCB10007s                             7
    AmycolatopsisdecaplaninaDSM44594s                             6
    ActinoplanesspN902109s                                        5
    ArthrobacteraurescensTC1s                                     5
    ActinosynnemamirumDSM43827NC0130931s                          4
    Allokutzneriaalbatas                                          3
    ActinoplanesspSE50110s                                        3
    BifidobacteriumadolescentisATCC15703s                         3
    ClavibactermichiganensissubspmichiganensisNCPPB...            2
    Bifidobacteriumanimalissubsplactiss                           2
    ArthrobacterphenanthrenivoransSphe3s                          2
    ArthrobacterarilaitensisRE117s                                2
2   ArthrobacterarilaitensisRE117s                                1
    ClavibactermichiganensissubspmichiganensisNCPPB...            1
    CitricoccusspCH26s                                            1
    CatenulisporaacidiphilaDSM44928s                              1
    Bifidobacteriumanimalissubsplactiss                           1
    BifidobacteriumadolescentisATCC15703s                         1
    ArthrobacterphenanthrenivoransSphe3s                          1
    ArthrobacterchlorophenolicusA6s                               1
    ArthrobacteraurescensTC1s                                     1
    ActinoplanesspN902109s                                        1
    AmycolatopsisorientalisHCCB10007s                             1
    AmycolatopsismediterraneiU32s                                 1
    AmycolatopsisdecaplaninaDSM44594s                             1
    Allokutzneriaalbatas                                          1
    ActinosynnemamirumDSM43827NC0130931s                          1
    ActinoplanesspSE50110s                                        1
1   CorynebacteriumdiphtheriaeNCTC13129s                          1
    CitricoccusspCH26s                                            1
2   CorynebacteriumdiphtheriaeNCTC13129s                          1

In [59]:
DF_copies = DF1[["fam","genome_name","id_gene"]].drop_duplicates().value_counts(["fam", "genome_name"]).to_frame("copy_number").reset_index()
DF_copies
#####################Observaciones###########
#DF_copies muestra el número de copias en cada genoma por familia de enzimas (no por semilla).
#Agregar una columna extra con la lista de los id de los genes que 
#corresponden a las copias que se están contando

,fam,genome_name,copy_number
0,1,CatenulisporaacidiphilaDSM44928s,9
1,1,AmycolatopsismediterraneiU32s,8
2,1,ArthrobacterchlorophenolicusA6s,8
3,1,AmycolatopsisorientalisHCCB10007s,7
4,1,AmycolatopsisdecaplaninaDSM44594s,6
5,1,ActinoplanesspN902109s,5
6,1,ArthrobacteraurescensTC1s,5
7,1,ActinosynnemamirumDSM43827NC0130931s,4
8,1,Allokutzneriaalbatas,3
9,1,ActinoplanesspSE50110s,3


**Función para obtener media y desviación estandar del número de copias**

Argumento: dataframe con fam, genome_name, copy_number.

Salida: Una lista con los umbrales para determinar a partir de cuántas copias se considera que hay expansión.
Hay que tomar en cuenta que el elemento 0 de la lista corresponde a la familia 1 (los valores están desplazados).

In [63]:
##El umbral se comentó el umbral en media+std (el artículo dice que habría que poner media+2*std). 

## Ya que la salida de la prueba de evo parece solo utilizar la media. En la prueba se 
## eligen genomas que contengan al menos 5 copias, es este umbral el que utilizaremos. 

def exp_threshold(data):
    F = data["fam"].drop_duplicates().value_counts().sum() #Número de familias distintas
    threshold = []
    for i in range(1,F+1): 
        mean = data[data["fam"]==str(i)]["copy_number"].mean() #Cantidad promedio de copias por genoma
        #std = data[data["fam"]==str(i)]["copy_number"].std() #Desviación estandar del número de copias por genoma
        #threshold.append(mean+std)
        threshold.append(mean)
    return threshold
exp_threshold(DF_copies)

[4.176470588235294, 1.0]

**Filtrar DF_copies para que solo muestre aquellos genomas que pasan el umbral. Es decir, aquellos que 
formaran las familias expandidas**

Argumento: DataFrame DF_copies y la lista de threshold
Salida: Lista con los genomas en los que se observó expansión. 

In [70]:
def exp_genomes(data, thresholds):
    F = data["fam"].drop_duplicates().value_counts().sum() #Número de familias distintas
    fam_list = list(range(1,F+1))
    L = len(data)
    #exp_genomes = []
    exp_genomes_dic = {}
    for f in fam_list:            ##Se puede optimizar con memoization
        for i in range(L):
            if ((data["fam"][i] == str(f)) & (data["copy_number"][i]>thresholds[f-1])):
                #exp_genomes.append(data["genome_name"][i])
                exp_genomes_dic[data["genome_name"][i]] = data["copy_number"][i]
    return exp_genomes_dic

exp_genomes(DF_copies,exp_threshold(DF_copies))
        
            
            
        
    

{'CatenulisporaacidiphilaDSM44928s': 9,
 'AmycolatopsismediterraneiU32s': 8,
 'ArthrobacterchlorophenolicusA6s': 8,
 'AmycolatopsisorientalisHCCB10007s': 7,
 'AmycolatopsisdecaplaninaDSM44594s': 6,
 'ActinoplanesspN902109s': 5,
 'ArthrobacteraurescensTC1s': 5}